# 用降采样提高绘图速度

当使用显示一条拥有大量数据点的曲线时，绘图速度会明显降低。由于屏幕的分辨率有限，绘制大量的线段并不能增加图表显示的信息，因此一般在显示大量数据时都会对其进行降采样运算。由于这种运算需要对数组中的每个元素进行迭代，因此需要使用编译语言提高运算速度。本节首先介绍如何使用Numba实现快速降采样算法，然后使用它在数据显示范围改变时更新Plotly图表中曲线。

In [8]:
import numba
import numpy as np

@numba.jit(nopython=True)
def _get_peaks(x, y, xr, yr, n, index0, index1):
    x0, x1 = x[index0], x[index1]
    dx = (x1 - x0) / n

    i = index0
    x_min = x_max = x[i]
    y_min = y_max = y[i]
    x_next = x0 + dx

    j = 0

    while True:
        xc = x[i]
        yc = y[i]
        if xc >= x_next or i == index1:
            if x_min > x_max:
                x_min, x_max = x_max, x_min
                y_min, y_max = y_max, y_min
            xr[j], xr[j + 1] = x_min, x_max
            yr[j], yr[j + 1] = y_min, y_max
            j += 2

            x_min = x_max = xc
            y_min = y_max = yc
            x_next += dx
            if i == index1:
                break
        else:
            if y_min > yc:
                x_min, y_min = xc, yc
            elif y_max < yc:
                x_max, y_max = xc, yc
        i += 1
    return j

下面对上面的`_get_peaks()`进行包装，使其更容易使用：

In [9]:
def get_peaks(x, y, n, x0=None, x1=None):
    x0 = x[0] if x0 is None else x0
    x1 = x[-1] if x1 is None else x1

    index0, index1 = np.searchsorted(x, [x0, x1])
    index1 = min(index1, len(x) - 1)
    xr = np.empty(2 * n)
    yr = np.empty(2 * n)
    length = _get_peaks(x, y, xr, yr, n, index0, index1)
    return xr[:length], yr[:length]

下面创建两条曲线的数据，每条曲线上有1万个点：

In [10]:
import numpy as np

def make_noise_sin_wave(period, phase, n):
    np.random.seed(42)

    x = np.random.uniform(0, 2*np.pi*period, n)
    x.sort()
    y = np.sin(x + phase)
    m = int(n*0.01)
    y[np.random.randint(0, n, m)] += np.random.randn(m) * 0.4
    return x, y

x1, y1 = make_noise_sin_wave(10, 0, 10000)
x2, y2 = make_noise_sin_wave(10, np.pi*0.5, 10000)

下面用`%timeit`测试`get_peaks()`的计算效率。第一次运行该函数时，Numba会对其进行JIT编译。

In [11]:
get_peaks(x1, y1, 500)
%timeit get_peaks(x1, y1, 500)

10000 loops, best of 3: 122 µs per loop


Here we use set `relayout_callback` argurment of `PlotlyWidget`, this will enable Plotly to send `plotly_relayout`  event data to the Python callback function. 

`PlotlyWidget.send(code, data)` send Javascript `code` and Json `data` to Plotly, and the `code` will be executed with the `data`.

In [12]:
from plotlyhelp import init_plotly_offline_mode
init_plotly_offline_mode()

下面的`demo()`返回包含两个控件的`HBox`对象其中的图表在显示范围改变时会调用`update_curve()`，在该函数中调用`get_peaks()`计算X轴显示范围之类的数据曲线，并调用`widget.send()`将曲线数据和Javascript代码发送给客户端。为了方便用户调试，在左侧的文本框中显示该函数接收到的数据。

In [13]:
from plotlyhelp import PlotlyWidget
from ipywidgets import Textarea, HBox
import json

def demo(count):
    xp1, yp1 = get_peaks(x1, y1, count)
    xp2, yp2 = get_peaks(x2, y2, count)

    line1 = {"x":xp1, "y":yp1, "name":"sin with noise", 
            "line":{"width":3, "color":"blue"}}
    line2 = {"x":xp2, "y":yp2, "name":"cos with noise", 
            "line":{"width":3, "color":"red"}}

    layout = {"title": "Update data from Python", "width":600, "height":400, "yaxis":{"fixedrange": True}}
    fig = {"data":[line1, line2], "layout":layout}

    def update_curve(data):
        textarea.value = json.dumps(data, indent=2)
        if all(not key.startswith("xaxis") for key in data):
            return
        xmin = data.get("xaxis.range[0]", None)
        xmax = data.get("xaxis.range[1]", None)
        if xmin is None or xmax is None:
            xmin, xmax = data.get("xaxis.range", [None, None])

        data = []
        for i, (x, y) in enumerate(zip([x1, x2], [y1, y2])):
            xp, yp = get_peaks(x, y, count, xmin, xmax)
            data.append({"x":xp, "y":yp})

        code = """
            for(i=0;i<graph.data.length;i++){
                graph.data[i]["x"] = data[i]["x"];
                graph.data[i]["y"] = data[i]["y"];
            }
        """

        widget.send(code, data)

    textarea = Textarea()
    widget = PlotlyWidget(fig, relayout_callback=update_curve)
    return HBox([textarea, widget])

下面显示降取样点数为500的曲线：

In [14]:
demo(500)

![使用降取样快速绘制曲线](plotly02.png)